NOTE : Considering some of GEE's user limits and avoiding some system errors, such as "ERROR: user memory limit exceeded","TimeoutError", etc,we will process or download some data beforehand and upload it to GEE Assets, where it will be called up when needed.

Special reminder: the following variables or parameters need to be modified

Must be amended : "roi", "region", "elevation", "dataset_id", "assetID", "filtered_samples"

May have to be modified (Depending on your actual situation) : 

In [ ]:
#### If you are trying to use geemap in coutries where Gooogle Services are blocked (e.g., China), 
#### you will need a VPN,then replace "10809" with your "proxy port number"to connect to Earth Engine servers.
#### Otherwise, you might encounter a connection timeout issue.

import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
#### Initializing GEE

import geemap
import ee
Map=geemap.Map()
Map

In [ ]:
#### Setting the boundaries of the study area.
#### Format: ee.Geometry.Rectangle(minLng, minLat, maxLng, maxLat)

roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
#### Defining variables
region = 'wuhan'

#### Calls for elevation data, which he has downloaded to GEE's Assets in "00--preparation.ipynb"
elevation = ee.Image("users/311605001111/hillshade_wuhan")

#### Visualisation parameters for Landsat8 images
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

# Define the relevant functions

In [ ]:
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('(B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7)/10000',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi)

def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

def maskSR_reverse(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    image_cloud = img.updateMask(mask.remap([0,1],[1,0]))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image_shadow = img.updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True).remap([0,1],[1,0]))
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    image_outlier = img.updateMask(bands.gt(0).rename('outlier'))
    return ee.ImageCollection([image_cloud,image_shadow,image_outlier]).sum()


In [ ]:
## img refers to an image that has been indexed but not cloud-masked
def classified_image(img):
    image = maskSR(img).select(bands).classify(trainedClassifier).eq(1).remap([0,1],[1,2]).rename('waterclass').float()
    invalidPixel = maskSR_reverse(img).select('pixel_qa').gt(0).remap([0,1],[1,0]).rename('waterclass').float()
    class_image = ee.ImageCollection([invalidPixel,image]).sum()
    invalidPixels = class_image.eq(0).multiply(ee.Image.pixelArea()).divide(1e6)
    invalidarea = invalidPixels.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 500,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    region = class_image.gte(0).multiply(ee.Image.pixelArea()).divide(1e6)
    regionarea = region.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 500,'maxPixels': 1e14,'tileScale': 2}).get('waterclass')
    rate = ee.Number(invalidarea).divide(ee.Number(regionarea)).multiply(100)
    return class_image.set({'system:id':img.get('system:id')}).set({'CLOUD_COVER':img.get('CLOUD_COVER')}).set({'invalid_percentage':rate})

def waterArea(image):
    classified_image = image.select('waterclass').eq(2).rename('waterclass')
    water_area = classified_image.multiply(ee.Image.pixelArea()).divide(1e6)
    waterarea = water_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': roi,
        'scale': 200,
        'maxPixels': 1e14,
        'tileScale': 2,
    })
    return image.set({'waterarea': waterarea.get('waterclass')})

def occurrence_Histogram(class_image):
    water = class_image.eq(2).selfMask()
    no_data = class_image.eq(0).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence')
    occurrence_water = occurrence.updateMask(water)
    occurrence_no_data = occurrence.updateMask(no_data)
    occurrence_HistogramCount = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.histogram(100,1),
        'geometry': roi,
        'scale': 30,
        'bestEffort': True,
        'tileScale': 2,
    })
    return class_image.set({'occurrence_HistogramCount': occurrence_HistogramCount.get('occurrence')})

def AutomaticCorrection_threshold(class_image):
    histogram = ee.List(ee.Dictionary(class_image.get('occurrence_HistogramCount')).get('histogram'))
    bucketMeans = ee.List(ee.Dictionary(class_image.get('occurrence_HistogramCount')).get('bucketMeans'))
    count_threshold = ee.Number(histogram.reduce(ee.Reducer.sum())).multiply(0.0017)
    index = histogram.map(lambda i : ee.Algorithms.If(ee.Number(i).gte(ee.Number(count_threshold)),ee.Number(i))).removeAll([None]).get(0)
    occurrence_threshold = bucketMeans.get(histogram.indexOf(index))
    return class_image.set({'occurrence_threshold':occurrence_threshold})

def AutomaticCorrection_enhanced(class_image):
    basemap = ee.Image.constant(0).toFloat().updateMask(class_image.gte(0)).rename('waterclass')
    water = class_image.eq(2).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence')
    occurrence_no_data = occurrence.updateMask(class_image.eq(0).selfMask())
    occurrence_threshold = class_image.get('occurrence_threshold')
    occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
    enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()
    return enhanced_water

# Water occurrence

You can view the download progress in "Google earth engine->Code Editor->Tasks"

"filtered_samples" denotes the training sample set downloaded to GEE's Assets in "01--Automatic training sample construction.ipynb",which needs to be adapted to your situation.

In [ ]:
# TIME = [['2000'],['2005'],['2010'],['2015'],['2020']]
TIME = [['2020']]
for time in TIME:
    print(time[0])
    startDate = time[0] + '-01-01'
    endDate = time[0] + '-12-31'
    l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)  
    l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)           
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
           .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)
    landsat_image = l8.merge(l7).merge(l5).map(water_index)
    
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    filtered_samples = ee.FeatureCollection('users/311605001111/' + region + '/' + region + '_9920')
    trainedClassifier = ee.Classifier.smileRandomForest(150).train(filtered_samples,label,bands)
    landsat_images = landsat_image.map(classified_image).filter(ee.Filter.gte('invalid_percentage',5)).filter(ee.Filter.lt('invalid_percentage',95))
    No_correct_image5 = landsat_image.map(classified_image).filter(ee.Filter.lt('invalid_percentage',5))
    No_correct_image6 = landsat_image.map(classified_image).filter(ee.Filter.gte('invalid_percentage',95))
    probable_correct_image = landsat_images.map(waterArea).filter(ee.Filter.gt('waterarea',5))
    No_correct_image1 = landsat_images.map(waterArea).filter(ee.Filter.lte('waterarea',5))
    occurrence_threshold = probable_correct_image.map(occurrence_Histogram).filter(ee.Filter.neq('occurrence_HistogramCount',None)).map(AutomaticCorrection_threshold)
    correct_image = occurrence_threshold.filter(ee.Filter.gt('occurrence_threshold',5)).filter(ee.Filter.lte('occurrence_threshold',75))
    No_correct_image2 = occurrence_threshold.filter(ee.Filter.lte('occurrence_threshold',5))
    No_correct_image3 = occurrence_threshold.filter(ee.Filter.gt('occurrence_threshold',75))
    No_correct_image4 = probable_correct_image.map(occurrence_Histogram).filter(ee.Filter.eq('occurrence_HistogramCount',None))
    correct_image = correct_image
    # print('Number of images to be corrected : ',correct_image.size().getInfo())
    No_correct_image = No_correct_image1.merge(No_correct_image2).merge(No_correct_image3).merge(No_correct_image4).merge(No_correct_image5).merge(No_correct_image6)
    # print('Number of images not to be corrected : ',No_correct_image.size().getInfo())
    
    #### Calculation of water occurrence
    if correct_image.size().getInfo() == 0 :
        No_Correct_waterPixel = No_correct_image.map(lambda i : i.select('waterclass').eq(2)).sum()
        No_Correct_validPixel = No_correct_image.map(lambda i : i.select('waterclass').gt(0)).sum()
        waterfrequency = No_Correct_waterPixel.divide(No_Correct_validPixel).rename('frequency')
    else:
        correct_waterPixel = correct_image.map(AutomaticCorrection_enhanced).sum()
        correct_validPixel = correct_image.map(lambda i : i.select('waterclass').gte(0)).sum()
        No_Correct_waterPixel = No_correct_image.map(lambda i : i.select('waterclass').eq(2)).sum()
        No_Correct_validPixel = No_correct_image.map(lambda i : i.select('waterclass').gt(0)).sum()
        waterPixel = ee.ImageCollection([correct_waterPixel,No_Correct_waterPixel]).sum()
        validPixel = ee.ImageCollection([correct_validPixel,No_Correct_validPixel]).sum()
        waterfrequency = waterPixel.divide(validPixel).rename('frequency')
    # Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"water frequency")
    
    #### The data is stored in GEE's Assets
    ## "dataset_id" indicates the file name; "assetID" indicates the file path in GEE's Assets.
    ## They need to be modified to suit your situation
    dataset_id = region + '_WF_' + time[0]
    assetID = 'users/311605001111/WF/' + dataset_id
    task = ee.batch.Export.image.toAsset(**{
        'image': waterfrequency,
        'description': dataset_id,
        'assetId': assetID,
        'scale': 30,
        'region': roi,
        'maxPixels': 1e13,
    })
    task.start()

# Appendix--download

By default, all processing results are saved to GEE's Assets. 

If required, they can also be downloaded to a local disk or Google Drive

## Download to Google Drive

In [ ]:
#### Calling the image data stored in GEE's Assets. 
## "data_id" indicates the image id stored in GEE's Assets

data_id = 'users/311605001111/WF/wuhan_WF_2020'
waterfrequency = ee.Image(data_id)

In [ ]:
#### The data will be stored in Google Drive
## "dataset_id" indicates the file name; "folder" indicates the file path in Google Drive.
## They need to be modified to suit your situation

year = 2020
region = 'wuhan'
dataset_id = region + '_WF_' + str(year)
folder = region + '_WF'

task = ee.batch.Export.image.toDrive(**{
    'image': waterfrequency,
    'description': dataset_id,
    'folder': folder,
    'scale': 30,
    'region': roi,
    'maxPixels': 1e13,
})
task.start()

## Download to local disk

In [ ]:
#### Calling the image data stored in GEE's Assets. 
## "data_id" indicates the image id stored in GEE's Assets

data_id = 'users/311605001111/WF/wuhan_WF_2020'
waterfrequency = ee.Image(data_id)

In [ ]:
#### The data will be stored in local dish
## "dataset_id" indicates the file name; "folder indicates the file path in local dish.
## For example, the path for this experiment is "D:/wuhan_WF/wuhan_WF_2020.tif"
## They need to be modified to suit your situation

year = 2020
region = 'wuhan'
folder = 'D:/' + region + '_WF/'+ region + '_WF_' + str(year) + '.tif'

task = geemap.ee_export_image(**{
    'ee_object' : waterfrequency,
    'filename' : folder, 
    'scale' : 30, 
    'region' : roi, 
    'file_per_band' : False        
})
task